## Demo of the distributed association scan hammer (DASH)

Source: Section 4 of *Secure multi-party linear regression at plaintext speed*

https://github.com/jbloom22/DASH

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import t
from statsmodels.api import OLS

### SIMULATE data

In [ ]:
np.random.seed(0)

K = 3
M = 10000

# Alice
N1 = 1000
y1 = np.random.randn(N1)
X1 = np.random.randn(N1, M)
C1 = np.random.randn(N1, K)

# Bob
N2 = 2000
y2 = np.random.randn(N2)
X2 = np.random.randn(N2, M)
C2 = np.random.randn(N2, K)

# Carla
N3 = 1500
y3 = np.random.randn(N3)
X3 = np.random.randn(N3, M)
C3 = np.random.randn(N3, K)

### PRIVATE COMPUTATION - Compress

In [ ]:
# Alice
yy1 = y1.T @ y1
Xy1 = X1.T @ y1
XX1 = (X1**2).sum(axis=0)

Cty1 = C1.T @ y1
CtX1 = C1.T @ X1

_, R1 = np.linalg.qr(C1)

# Bob
yy2 = y2.T @ y2
Xy2 = X2.T @ y2
XX2 = (X2**2).sum(axis=0)

Cty2 = C2.T @ y2
CtX2 = C2.T @ X2

_, R2 = np.linalg.qr(C2)

# Carla
yy3 = y3.T @ y3
Xy3 = X3.T @ y3
XX3 = (X3**2).sum(axis=0)

Cty3 = C3.T @ y3
CtX3 = C3.T @ X3

_, R3 = np.linalg.qr(C3)

### MULTI-PARTY COMPUTATION - Combine

Computation is now independent of the sample sizes. Practical security follows from non-invertibility of compression. For theoretical guarantees, do the following with secure multi-party computation.

In [ ]:
D = N1 + N2 + N3 - K - 1

yy = yy1 + yy2 + yy3
Xy = Xy1 + Xy2 + Xy3
XX = XX1 + XX2 + XX3

Cty = Cty1 + Cty2 + Cty3
CtX = CtX1 + CtX2 + CtX3

_, R = np.linalg.qr(np.concatenate([R1, R2, R3]))
invR = np.linalg.inv(R)

Qty = invR.T @ Cty 
QtX = invR.T @ CtX

QtyQty = Qty.T @ Qty
QtXQty = QtX.T @ Qty
QtXQtX = (QtX**2).sum(axis=0)

yyq = yy - QtyQty
Xyq = Xy - QtXQty
XXq = XX - QtXQtX

The endpoints of secure computation are effect size and squared standard error:

In [ ]:
beta = Xyq / XXq
sigma_sq = (yyq / XXq - beta**2) / D

With these we can compute t statistic and p-value:

In [ ]:
sigma = np.sqrt(sigma_sq)
tstat = beta / sigma
pval = 2 * t.cdf(-abs(tstat), D)

Organize the DASH results above in a dataframe:

In [ ]:
df_dash = pd.DataFrame({'beta': beta,
                        'sigma': sigma, 
                        'tstat': tstat, 
                        'pval': pval})

df_dash

### VERIFY correctness

Recompute using the OLS model from statsmodel API:

In [ ]:
M0 = 5 # verify the first 5

y = np.concatenate([y1 ,y2, y3])
X = np.concatenate([X1, X2, X3])
C = np.concatenate([C1, C2, C3])

res = np.zeros([M0, 4])
for m in range(M0):
    model = OLS(y, np.concatenate((X[:, m:m+1], C), axis=1), hasconst=False)
    tmp_res = model.fit()
    res[m] = np.array([tmp_res.params[0], tmp_res.bse[0], tmp_res.tvalues[0], tmp_res.pvalues[0]]).ravel()

df_ols = pd.DataFrame({'beta': res[:,0],
                       'sigma': res[:,1],
                       'tstat': res[:,2],
                       'pval': res[:,3]})

Verify agreement up to 10 digits after the decimal point.

In [ ]:
df_dash = df_dash.apply(lambda x: round(x, 10))
df_ols = df_ols.apply(lambda x: round(x, 10))
np.array(df_dash.iloc[:M0] == df_ols).all() # Returns TRUE